<img src="PEST++V3_cover.jpeg" style="float: left">

<img src="flopylogo.png" style="float: right">

<img src="AW&H2015.png" style="float: center">

# History match the Freyberg model using a single ``K`` parameter

Freyberg (1988) was notable for the discussing what is often called "point calibration" and "overfitting". It is a heterogenous 2D areal (1-layer) model that is a step up in complexity from our xsec model. Recall it looks like this, as shown in the original Freyberg (1988) paper on the left, and a Groundwater Vistas version on the right (from the file in the GW_Vistas subdirectory).   

<img src="Freburg1988_fig1.png" style="float: left">
<img src="Freyberg_k_plot_GW_Vistas.png" style="float: right">

As we discussed in our first notebook, one way to get to an optimal model is to start simple and add complexity. To do this righte we need to add our forecasts first along with the calibration targets, and look at both as more model complexity is added.  One of the findings of the Freyberg (1988) model is that some students made their model too complex, which diminished its performance for the prediction of interest. Starting simple will keep us out of danger (at least for this notebook). 

This version of the Freyberg model has 3 stress periods:  1 steady state -> 1 transient for 5 years -> 1 steady state

The river stage and node conductance changes in each stress period (see .riv file)

The recharge starts out wetter in stress period one, then is drier in stress periods 2 and 3 (see .rch file)

Pumping well discharge changes each stress period (see .wel file)

In addition to heads and fluxes, MODPATH is also included in the PEST++ model run file to perform particle tracking after MODFLOW finishes

***********************************************
__So, the objectives of this notebook are to:__

1) Ease you into the Freyberg model as we'll be using it for the rest of the class

2) Revisit the PEST control (pst) file

3) Look at typical summary statistics and plots the describe our degree of fit

4) Look at how head data constraints ripple to different forecast types

## Standard two blocks to prep the notebook

In [ ]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import pyemu

In [ ]:
import freyberg_setup as fs
pst_name = fs.PST_NAME_UN
working_dir = fs.WORKING_DIR_UN
fs.setup_pest_un()
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = 0
pst.observation_data.loc[pst.observation_data.obgnme=='calflux', 'weight']=0

pst.write(os.path.join(working_dir,pst_name))


In [ ]:
fs.plot_model(working_dir, pst_name)

## We've given you all the files you need to run PEST++ on this model.  

### Open up the .pst file, and orient yourself using to the PDF file "Annotated_PEST_control_file_SIR20105169.pdf".  We've highlighted the PEST variables that you are likely to manipulate; *NOTE: that unless something is in [brackets] a value must be supplied, but the default values you see in our example .pst files will be fine.*  

### To guide your eyes through the PEST control file, answer these questions:

1) How many parameters are we running? 

2) How many are adjustable? 

3) How many types of observations are included?

4) How many forecasts? What types?

5) How many template (tpl) files do we have?

6) How many instruction (ins) files do we have? 

## Now that you know the .ins, and .tpl files, open them in a text editor (make sure you are looking at the ones in /activities/freyberg_k subdirectory). In a seperate terminal window, run TEMPCHEK, INSCHEK and PESTCHEK on the files we've given you. 

### Okay, you've got running PEST utilities in a separate terminal window by now?  To speed things up we've given you a way to execute these utilities from within the Freyberg notebook.  We've included the equivalent version of what you just did in the next three code blocks.  Execute the code block then look at terminal window where you launched this notebook.

In [ ]:
pyemu.helpers.run("tempchek hk_layer_1.ref.tpl",cwd=working_dir)

In [ ]:
pyemu.helpers.run("inschek flux.dat.ins flux.dat",cwd=working_dir)

In [ ]:
pyemu.helpers.run("inschek freyberg.hyd.bin.dat.ins freyberg.hyd.bin.dat",cwd=working_dir)

### Whoa - lots of heads!  Observations don't cost us any runs/computer time, so from now on we'll be carrying many more observations. 

In [ ]:
pyemu.helpers.run("inschek freyberg.travel.ins freyberg.travel",cwd=working_dir)

In [ ]:
pyemu.helpers.run("inschek vol.dat.ins vol.dat",cwd=working_dir)

In [ ]:
pyemu.helpers.run("pestchek {0}".format(pst_name),cwd=working_dir)

# Okay, let's run this thing. 

## Because we call a program from within the Jupyter Notebook you have to look at the terminal window that you used to start the notebook to see the screen report of the run.  So, when executing this next block look at your terminal window to see the run.  It will say "Simulation complete..." when finished.

### NOTE:  And/or wait until the standard out  reports a "0" below this next block (=when the run is finished) before going on.

In [ ]:
pyemu.helpers.run("pestpp {0} ".format(pst_name),cwd=working_dir)

## ``PEST++`` only ran the model one time - why?

Yeah, that's right, the NOPTMAX=0 thing again.  We had that set to zero because we want to check the plumbing before burning the silicon. Did everything run (i.e., did you see "Simulation Complete..." in your terminal window?  Like before, you *could* change NOPTMAX to 20 in a text editor.  But, pyemu can do it for you with the next block.  

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = 20
pst.write(os.path.join(working_dir,pst_name))

#### "Trust but verify"....by running PESTCHEK

In [ ]:
pyemu.helpers.run("pestchek {0} ".format(pst_name),cwd=working_dir)

### NOPTMAX=0 warning is gone. Now we let's run it.  Just like before  you have to look at the terminal window that you used to start the notebook to see the screen report of the run.  So, when executing this next block look at your terminal window to see the run.  It will say "Simulation complete..." when finished.

Or wait until the standard out  reports a "0" below this next block (=when the run is finished) before going on.

In [ ]:
pyemu.helpers.run("pestpp {0} ".format(pst_name),cwd=working_dir)

### Let's explore the results

First let's look at the measurement objective function (Phi), which is calculated using the sum of squared weighted residuals.   

In [ ]:
df_obj = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".iobj")),index_col=0)
df_obj

In [ ]:
df_obj.loc[:,["total_phi","model_runs_completed"]].plot(subplots=True)

### Look at the FINAL OPTIMISATION RESULTS in the terminal where PEST++ ran (you can also find it near the bottom of the .rec file).  Which target group(s) matter?  How was splitting the contributions to PHI accomplished in the PEST control file?

For this problem, recall our objective function is calculated using this equation:


<img src="SOSWR_eq_AW&H2015.png" style="float: center">

where Phi is the "sum of squared weighted residuals" that we look to minimize, *whi* is the weight for the ith head observation; *hm* is the measured (observed) head target; *hs* is the simulated head; and n is the number of observations.  If we use only heads for calibration the PHI only reflects the sum of squared weighted residuals for the observed-simulated head targets. 

# Hey, we told PEST to try 20 parameter estimation upgrades but it stopped at 3!  What gives?!?

(hint: search the .rec file for OPTIMIZATION COMPLETE)

PEST and PEST++ will quit the parameter estimation process if one of these 4 conditions is met:

1) The maximum number of interations specified in NOPTMAX is reached

2) The fit is not getting any better based on a user-supplied closure

3) The parameters are not changing based on a user-supplied closure

4) The user killed the run, usually with a ctrl-c  (happens quite frequently)

##  Let's evaulate our fit using the observed-simulated residuals


In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
res_nz = pst.res.loc[pst.nnz_obs_names,:]
res_nz

In [ ]:
pyemu.plot_utils.res_1to1(pst)

### Not too shabby!  Thanks PEST++.

### We can also look at the residual (y-axis) compared to the observation magnitude (x-axis).  The closer the circle is to the black line the better the fit.  The mean residual is shown as a red line, the pink zone is 1 standard deviation on either side of the mean.

### But we had a lot of other observations listed in the PEST control file.  What do they look like?

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.res

## Observations with 1970 in them represent the calibration period; those with 1975 represent the forecast period.  The last column is the weight given to them in the parameter estimation. 

# Now let's look at what the calibration did for uncertainty reduction

### First, let's look the change in uncertainty for our one horizontal hydraulic conductivity (Kh) parameter

In [ ]:
df_paru = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".par.usum.csv")),index_col=0)
df_paru

#### NOTE: Recall that because we log transformed the Kh parameter the uncertainty results are reported as logarithms in the dataframe above.  What you'll see in the MODFLOW input file is the non-log transformed Kh value, which is 10^0.693406 = 4.9465457 (prior mean) or 10^0.833 = 6.818 (posterior mean)

## Now let's plot it:

In [ ]:
ax = pyemu.helpers.plot_summary_distributions(df_paru,label_post=True)
ax.set_xlabel("$log_{10}(\\frac{L}{T})$")

### The dotted gray line represents the "prior" information as expressed by the parameter bounds in the PEST control file.  The shaded area is the uncertainty after the calibration.



# Now let's look at changes in model forecast uncertainty, first as a table then as a plot

(The forecasts of interest are identified by putting "fore" in the observation group name, so they are __foreflux, forehead, and foretrav__)

In [ ]:
df_predu = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".pred.usum.csv")),index_col=0)
df_predu

In [ ]:
figs, axes = pyemu.helpers.plot_summary_distributions(df_predu,subplots=True)


### And  by comparing prior to posterior standard deviations

In [ ]:
for forecast in df_predu.index:
    ax = df_predu.loc[forecast,["prior_stdev","post_stdev"]].plot(kind="bar")
    ax.set_title(forecast)
    plt.show()

# Wow!  The calibration really reduced the uncertainty in our forecasts! So we can call it a day and bill the client?

### Hmm, maybe not yet...the reductions look suspiciously similar.  Let's use a summary statistic, percent reduction in uncertainty, to help be quantitative on the reduction in forecast uncertainty

In [ ]:
df_predu.loc[:,"percent_reduction"] = 100.0 * (1.0 - (df_predu.post_stdev / df_predu.prior_stdev))
df_predu.percent_reduction

## Does it make sense that the travel time uncertainty would be reduced in the same proportion as the head forecasts?

## Just for fun, and because we can, let's look again at forecast uncertainty with the "truth"...

In [ ]:
figs, axes = pyemu.helpers.plot_summary_distributions(df_predu,subplots=True)
for ax in axes:
    fname = ax.get_title().lower()
    ylim = ax.get_ylim()
    v = pst.observation_data.loc[fname,"obsval"]
    ax.plot([v,v],ylim,"k--")
    ax.set_ylim(ylim)

# Uh oh!  Why are some forecasts not bracketed by the posterior distribution? This means uncertainty analysis "failed"! The dashed line of the uncalibrated model (what we call the "Prior") is more reliable - why?